# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside your personal course repository for the course 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.


In [22]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy
import unittest

In [23]:
Nimply = namedtuple("Nimply", "row, num_objects")
Move = namedtuple("Move", "row num_objects fitness")

import numpy as np

#### NIM Game

In [24]:
POPULATION_SIZE = 10
LAMBDA = 20
N_GENERATIONS = 50

In [25]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self):
        return self._rows
    
    @property
    def k(self) -> int:
        return self._k

    def nimming2(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        if self._k is not None:
            num_objects = min(num_objects, self._k)
        self._rows[row] -= num_objects


In [26]:
def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


In [27]:
def select_parent(population, tournament_size=2):
    return min(random.choices(population, k=tournament_size), key=lambda i: i.fitness)

def evolve_one_plus_lambda(parent, mutation_rate, state):
    child = mutation(parent, state)
    return child

def mutation(parent, nim):
    if parent is None:
        # Se il genitore è None, genera una mossa casuale
        row = random.choice([r for r, c in enumerate(nim.rows) if c > 0])
        num_objects = random.randint(1, nim.rows[row])
        a = nim_sum(nim)
        offspring = Move(row, num_objects, a)
    else:
        if nim.k is None:
            elements = random.randrange(1, nim.rows[parent.row] + 1)
        else:
            elements = min(nim.k, random.randrange(1, nim.rows[parent.row] + 1))
        temp_rows = nim.rows.copy()
        temp_rows[parent.row] -= elements 
        offspring = Move(parent.row, elements, nim_sum(temp_rows))

    return offspring

#### Player

## Strategies:
    - Pure Random
    - Optimal
    - Evolvable based on ES (1+λ)

In [28]:
class Player:
    def __init__(self, strategy) -> None:
        assert strategy in ['optimal', 'pure_random', 'evolvable_based_on_ES_1_lambda'], f"Strategy non-available"
        self._strategy = strategy

    def moves(self, Nim, alpha = 0.5, beta = 0.5):
        if self._strategy == 'evolvable_based_on_ES_1_lambda':
            return self.evolvable_based_on_ES_1_lambda(Nim, alpha)
        elif self._strategy == 'optimal':
            return self.optimal(Nim)
        elif self._strategy == 'pure_random':
            return self.pure_random(Nim)
        else: 
            assert f"Can't use this strategy"
        return
    
    def pure_random(self, state: Nim) -> Nimply:
        """A completely random move"""
        nonzeroind = np.nonzero(state.rows)[0]
        random_row = random.choice(nonzeroind)

        if state._k == None:
            random_elements = random.randint(1, state.rows[random_row])
        else:
            random_elements = random.randint(1, min(state._k, state.rows[random_row]))
        return Nimply(random_row, random_elements)
    
    def optimal(self, state: Nim) -> Nimply:
        analysis = analize(state)
        logging.debug(f"analysis:\n{pformat(analysis)}")
        spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
        if not spicy_moves:
            spicy_moves = list(analysis["possible_moves"].keys())
        ply = random.choice(spicy_moves)
        return ply
    
    def evolvable_based_on_ES_1_lambda(self, state: Nim, alpha = 0.5):
        # Popolazione iniziale di possibili mosse
        population = [mutation(None, state) for _ in range(POPULATION_SIZE)]

        for g in range(N_GENERATIONS):
            offspring = []

            for i in range(LAMBDA):
                # Selezione del genitore (nel tuo caso, potrebbe essere tramite il torneo)
                parent = select_parent(population)

                # Generazione del figlio mediante mutazione del genitore
                child = mutation(parent, state)

                # Aggiornamento del genitore con alpha
                parent.row = int(parent.row + alpha * (child.row - parent.row))
                parent.num_objects = int(parent.num_objects + alpha * (child.num_objects - parent.num_objects))

                # Aggiungi il figlio alla lista degli offspring
                offspring.append(child)

            # Unisci genitori e figli per formare la nuova popolazione
            population += offspring

            # Ordina la popolazione in base al fitness e seleziona i primi POPULATION_SIZE individui
            population = sorted(population, key=lambda i: i.fitness, reverse=False)[:POPULATION_SIZE]
            logging.debug(f"Actual best {population[0]}")

        # Restituisci la migliore mossa
        best_move = Nimply(population[0].row, population[0].num_objects)
        return best_move


#### Games 

In [29]:
def evaluate(agent_strategy, opponent_strategy, parameter_dict: dict = {"alpha": None, "beta": None}, NUM_MATCHES=100, random_board=True) -> float:
    won = 0
    start = 0

    for m in range(NUM_MATCHES):
        agent = Player(agent_strategy)
        opponent = Player(opponent_strategy)

        turn = start
        start = 1 - start

        if random_board:
            rows = random.randint(1, 10)
            K = random.randint(0, 2 * rows)
            if K == 0:
                K = None
        else:
            rows = 3
            K = 3

        nim = Nim(rows, K)

        game_over = [0 for _ in range(rows)]

        logging.debug(f"\n\n\n--------NEW GAME---------")

        while nim.rows != game_over:
            if turn == 0:
                logging.debug(f" Actual turn: Agent")
                if agent_strategy == 'evolvable_based_on_ES_1_lambda':
                    assert parameter_dict['alpha'] is not None, "Please provide parameter_dict for the evolvable strategy"
                    ply = agent.moves(nim, parameter_dict['alpha'])
                else:
                    ply = agent.moves(nim)
                logging.debug(f" \tAgent:   <Row: {ply.row}- Elements: {ply.num_objects}>")
            else:
                logging.debug(f" Actual turn: Opponent")
                if opponent_strategy == 'optimal':
                    # Implementa la logica per la strategia 'optimal'
                    ply = opponent.moves(nim)
                else:
                    ply = opponent.moves(nim)
                logging.debug(f" \tOpponent:   <Row: {ply.row}- Elements: {ply.num_objects}>")
            if ply.num_objects == 0:
                print(f"turn = {turn} ")
            nim.nimming(ply)
            logging.debug(f" \tTable after move: {nim} and Nim_sum: {nim_sum(nim)}\n")

            turn = 1 - turn

        logging.debug(f"--------GAME OVER---------")
        if turn == 1:
            won += 1
        else:
            logging.debug(f"Game Lost by the agent is the n°{m}")

    return won / NUM_MATCHES


#### Evaluation

In [30]:
logging.getLogger().setLevel(logging.INFO)
parameter_dict= {}

# Insert here your own parameters
# Best values of alpha and beta against a pure random opponent are respectively 0.4 and 0.1 generally

parameter_dict["alpha"] = 0.5
parameter_dict["beta"] = 0.1
parameter_dict["alpha_opp"] = 0.99
parameter_dict["beta_opp"] = 0.1

# Evaluation Section:


# print(f"Agent Won: {evaluate(agent_strategy='evolvable', opponent_strategy='pure_random', parameter_dict = parameter_dict)*100}% of the games")
print(f"Agent Won: {evaluate(agent_strategy='evolvable_based_on_ES_1_lambda', opponent_strategy='pure_random', parameter_dict = parameter_dict)*100}% of the games")


AttributeError: 'list' object has no attribute 'rows'